In [1]:
import h5py    
import numpy as np    
import torch
from torch import nn
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import random
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
from SVWN3 import f_svwn3
import csv
import copy
from NN_models import NN_2_256, NN_8_256, NN_8_64

In [2]:
def ref(x, y):
    ''' 
    returns reference energies for points of a reaction grid from Reference_data.csv
    '''
    hartree2kcal = 627.5095
    with open("Reference_data.csv", newline='', encoding='cp1251') as csvfile:
        ref_file = csv.reader(csvfile, delimiter=",")
        k = 1
        if y == 391:
            k = hartree2kcal
        ref = []
        for n, i in enumerate(ref_file):
            if x <= n + 1 <= y:
                ref.append((i[0], float(i[2]) * k))

        return ref

def load_ref_energies():
    '''Returns {db_name: [equation, energy]}'''
    ref_e = { # Получение референсных энергий
        "MGAE109":ref(8, 116),
        "IP13":ref(155, 167),
        "EA13":ref(180, 192),
        "PA8":ref(195, 202),
        "DBH76":ref(251, 288) + ref(291, 328),
        "NCCE31":ref(331, 361),
        "ABDE4":ref(206, 209),
        # "AE17":ref(375, 391),
        "pTC13":ref(232, 234) + ref(237, 241) + ref(244, 248)
        } 
    return ref_e

def load_component_names():
    '''
    Returns {db_name: {id: {'Components': [...], 'Coefficients: [...]'
                                }
                            }
                        }
     which is a dictionary with Components and Coefficients data about all reactions
    '''
    with open("total_dataframe_sorted_final.csv", newline='', encoding='cp1251') as csvfile:
        ref_file = csv.reader(csvfile, delimiter=",")
        ref = dict()
        current_database = None
        
        for n, line in enumerate(ref_file):
            line = np.array(line)
            if n == 0:
                components = np.array(line)
            else:
                reaction_id = int(line[0])
                reaction_database = line[1]
                reaction_component_num = np.nonzero(list(map(float, line[2:])))[0] + 2
                if reaction_database in ref:
                    ref[reaction_database][reaction_id] = {'Components': components[reaction_component_num], 'Coefficients': line[reaction_component_num]}
                else: 
                    ref[reaction_database] = {reaction_id: {'Components': components[reaction_component_num], 'Coefficients': line[reaction_component_num]}}
        return ref

In [3]:
def get_compounds_coefs_energy_v2(reactions, energies):
    '''Returns {id: 
                    {'Components': [...], 'Coefficients: [...]', 'Energy: float', Database: str
                                }
                            }
    which is a dictionaty from load_component_names with Energy information added
    '''
    data_final = dict()
    i = 0
    databases = load_ref_energies().keys()
    for database in databases:
        data = reactions[database]
        for reaction in data:
            data[reaction]['Energy'] = energies[database][reaction][1]
            data_final[i] = {'Database': database,
                         'Components': reactions[database][reaction]['Components'],
                         'Coefficients': reactions[database][reaction]['Coefficients'].astype(np.float64),
                         'Energy': energies[database][reaction][1]
            
        }
            i += 1
        
    return data_final

In [4]:
def get_h5_names(reaction):
    '''reaction must be from the function get_compounds_coefs_energy_v2'''
    database_match = {
        'MGAE109': 'mgae109',
        'IP13': 'ip13',
        'EA13': 'ea13',
        'PA8': 'pa8',
        'DBH76': 'ntbh38',
        'NCCE31': 'ncce31',
        'ABDE4': 'abde4',
        'AE17': 'ae17',
        'pTC13': 'ptc13'
    }
    names = []
    for elem in reaction['Components']:
        database = database_match[reaction['Database']]
        names.append(f'{elem}.h5')
    return names

In [5]:
def add_reaction_info_from_h5(reaction):
    '''
    reaction must be from get_compounds_coefs_energy_v2
    returns merged descriptos array X, integration weights, 
    a and b densities and indexes for backsplitting
    
    Adds the following information to the reaction dict using h5 files from the dataset:
    Grid : np.array with grid descriptors (scaled with np.log1p)
    Weights : list with integration weights of grid points
    Densities : np.array with alpha and beta densities data for grid points
    HF_energies : list of Total HF energy (T+V) which needs to be added to E_xc
    backsplit_ind: list of indexes where we concatenate molecules' grids
    '''
    X = np.array([]) 
    backsplit_ind = []
    HF_energies = np.array([])
    for component_filename in get_h5_names(reaction):
        with h5py.File(component_filename, "r") as f:
            HF_energies = np.append(HF_energies, f["ener"][:][0])
            X_raw = np.array(f["grid"][:])
            if len(X) == 0:
                X = X_raw[:, 3:-1]
            else:
                X = np.vstack((X, X_raw[:, 3:-1]))
            backsplit_ind.append(len(X))
    densities = X[:, 1:3]
    weights = X[:,0]
    X = np.log1p(X[:, 1:])

    labels = ['Grid', 'Weights', 'Densities', 'HF_energies', 'backsplit_ind']
    values = [X, weights, densities, HF_energies, backsplit_ind]
    for label, value in zip(labels, values):
        reaction[label] = value

    return reaction

In [6]:
def make_reactions_dict():
    '''
    Returns a dict like {reaction_id: {*reaction info}} with all info available listed below:
    ['Database', 'Components', 'Coefficients', 'Energy', 'Grid', 'Weights', 'Densities', 'HF_energies', 'backsplit_ind']
    '''
    data = get_compounds_coefs_energy_v2(load_component_names(), load_ref_energies())
    for i in data.keys():
        data[i] = add_reaction_info_from_h5(data[i])

    return data
    

In [ ]:
def get_local_energies(reaction, constants, densities):
    calc_reaction_data = {}
    local_energies = np.array(list(map(f_svwn3, densities, constants)))
    calc_reaction_data['Local_energies'] = local_energies
    return calc_reaction_data


def add_calc_reaction_data(reaction, calc_reaction_data):
    calc_reaction_data['Weights'] = reaction['Weights']
    calc_reaction_data['Densities'] = reaction['Densities']
    return calc_reaction_data

def backsplit(reaction, calc_reaction_data):
    backsplit_ind = reaction['backsplit_ind']
    splitted_data = dict()
    stop = 0

    for i, component in enumerate(reaction['Components']):
        splitted_data[component] = dict()
        start = stop
        stop = backsplit_ind[i]
        for elem in ('Local_energies', 'Weights', 'Densities'):
            splitted_data[component][elem] = calc_reaction_data[elem][start:stop]
    return splitted_data


def integration(reaction, splitted_calc_reaction_data):
    molecule_energies = dict()
    for i, component in enumerate(reaction['Components']):
        molecule_energies[component] = np.sum(splitted_calc_reaction_data[component]['Local_energies'] \
                                              * (splitted_calc_reaction_data[component]['Densities'][:,0] \
                                              + splitted_calc_reaction_data[component]['Densities'][:,1]) \
                                              * (splitted_calc_reaction_data[component]['Weights'])) \
                                              + reaction['HF_energies'][i]
    return molecule_energies


def get_energy_reaction(reaction, molecule_energies):
    hartree2kcal = 627.5095
    s = 0
    for coef, ener in zip(reaction['Coefficients'], molecule_energies.values()):
        s += coef*ener
    reaction_energy_kcal = s * hartree2kcal
    return reaction_energy_kcal


def calculate_reaction_energy(reaction, constants):
    calc_reaction_data = get_local_energies(reaction, constants, reaction['Densities'])
    calc_reaction_data = add_calc_reaction_data(reaction, calc_reaction_data)

    splitted_calc_reaction_data = backsplit(reaction, calc_reaction_data)

    molecule_energies = integration(reaction, splitted_calc_reaction_data)
    
    reaction_energy_kcal = get_energy_reaction(reaction, molecule_energies)

    return reaction_energy_kcal

In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(42)

In [8]:
data = make_reactions_dict()

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'C_mgae109.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
def train_split(data, test_size, shuffle=False):
    if shuffle:
        keys = list(data.keys())
        random.shuffle(keys)
        for i in keys:
            data[keys[i]] = data[i]

    train, test = dict(), dict()
    border = round(len(data.keys()) * (1 - test_size))
    for i in range(len(data.keys())):
        if i <= border:
            train[i] = data[i]
        else:
            test[i] = data[i]
    return train, test


In [9]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data
        
    def __getitem__(self, i):
        return self.data[i], self.data[i]['Energy']
    
    def __len__(self):
        return len(self.data.keys())

test_size = 0.2

data_train, data_test = train_split(data, 0.2, True)


train_set = Dataset(data=data_train)
train_dataloader = torch.utils.data.DataLoader(train_set, 
                                               batch_size=1, 
                                               num_workers=2)

test_set = Dataset(data=data_test)
test_dataloader = torch.utils.data.DataLoader(test_set, 
                                              batch_size=1, 
                                              num_workers=2)

NameError: name 'train_split' is not defined

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')

model = NN_8_256().to(device)
model.load_state_dict(torch.load('predoptimized_2.param'))

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))
criterion = nn.L1Loss()

In [ ]:
def train(model, criterion, optimizer, train_loader, test_loader, n_epochs=20):
    train_loss_mae = []
    test_loss_mae = []


    for epoch in range(n_epochs):
        print(epoch+1)
        # train
        model.train()
        progress_bar = tqdm(train_loader)


        train_mae_losses_per_epoch = []
        for X_batch, y_batch in progress_bar:
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            predictions = np.exp1m(model(X_batch_grid))
            reaction_energy = calculate_reaction_energy(X_batch, predictions)
            loss = criterion(np.array(reaction_energy), np.array(y_batch))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_mae_losses_per_epoch.append(loss.item())
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        #test
        model.eval()
        test_mae_losses_per_epoch = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
                preds = np.exp1m(model(X_batch_grid))
                reaction_energy = calculate_reaction_energy(X_batch, predictions)
                loss = criterion(np.array(reaction_energy), np.array(y_batch))
                test_mae_losses_per_epoch.append(loss.item())
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))
        print(f'train MAE Loss = {train_loss_mae[epoch]:.8f}')
        print(f'test MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mse, train_loss_mae, test_loss_mse, test_loss_mae, preds[0].cpu().detach().numpy()


train_loss_mse, train_loss_mae, test_loss_mse, test_loss_mae, preds = train(model, criterion, optimizer, 
                                                                            train_dataloader,
                                                                            test_dataloader, n_epochs=20)